In [2]:
import torch
import torch.nn.functional as F

y = torch.tensor([1.0])  # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2]) # weight parameter
b = torch.tensor([0.0])  # bias unit

z = x1 * w1 + b          # net input
a = torch.sigmoid(z)     # activation & output

loss = F.binary_cross_entropy(a, y)
print(loss)

tensor(0.0852)


In [3]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b 
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [4]:
loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [18]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hidden1, num_hidden2):
        super().__init__()

        self.layers = torch.nn.Sequential(
            # first hidden layer
            torch.nn.Linear(num_inputs, num_hidden1),
            torch.nn.ReLU(),

            # second layer
            torch.nn.Linear(num_hidden1, num_hidden2),
            torch.nn.ReLU(),

            # output dense layer
            torch.nn.Linear(num_hidden2, num_outputs),

        )

    def forward(self, x):
        logits = self.layers(x)

        return logits

In [23]:
model = NeuralNetwork(7,3,5,3)
model

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=7, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=3, bias=True)
    (3): ReLU()
    (4): Linear(in_features=3, out_features=3, bias=True)
  )
)

In [24]:
model.parameters

<bound method Module.parameters of NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=7, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=3, bias=True)
    (3): ReLU()
    (4): Linear(in_features=3, out_features=3, bias=True)
  )
)>

In [25]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 70


In [26]:
print(model.layers[0].weight)


Parameter containing:
tensor([[ 0.3743, -0.2207, -0.3457,  0.3285,  0.1665, -0.0646,  0.2426],
        [-0.2337,  0.2816,  0.3588,  0.1443, -0.3617, -0.3216,  0.0085],
        [ 0.1468,  0.0848, -0.2713,  0.0798, -0.3146,  0.3751, -0.2396],
        [-0.1782,  0.2645, -0.1191,  0.2251,  0.0341, -0.2064,  0.1626],
        [-0.1428,  0.3391,  0.3387, -0.2205, -0.3140,  0.1903,  0.2570]],
       requires_grad=True)


In [27]:
print(model.layers[0].weight.shape)


torch.Size([5, 7])


In [29]:
torch.manual_seed(123)

X = torch.rand((1, 7))
out = model(X)
print(out)

tensor([[-0.3694,  0.2483, -0.1761]], grad_fn=<AddmmBackward0>)


In [30]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.3694,  0.2483, -0.1761]])


In [31]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.2458, 0.4559, 0.2983]])


In [32]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

In [33]:
X_train.shape, y_train.shape

(torch.Size([5, 2]), torch.Size([5]))

In [34]:
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [35]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]        
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [36]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

In [37]:
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [38]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [43]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [ 2.3000, -1.1000]]) tensor([0, 1])
Batch 2: tensor([[ 2.7000, -1.5000],
        [-0.5000,  2.6000]]) tensor([1, 0])


In [52]:
import torch.nn.functional as F


torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2, num_hidden1=8, num_hidden2=4)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels) # LOSS Function

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval()
    # Optional model evaluation

Epoch: 001/010 | Batch 000/002 | Train/Val Loss: 0.76
Epoch: 001/010 | Batch 001/002 | Train/Val Loss: 0.61
Epoch: 002/010 | Batch 000/002 | Train/Val Loss: 0.45
Epoch: 002/010 | Batch 001/002 | Train/Val Loss: 0.14
Epoch: 003/010 | Batch 000/002 | Train/Val Loss: 0.35
Epoch: 003/010 | Batch 001/002 | Train/Val Loss: 0.02
Epoch: 004/010 | Batch 000/002 | Train/Val Loss: 0.50
Epoch: 004/010 | Batch 001/002 | Train/Val Loss: 0.03
Epoch: 005/010 | Batch 000/002 | Train/Val Loss: 0.01
Epoch: 005/010 | Batch 001/002 | Train/Val Loss: 0.28
Epoch: 006/010 | Batch 000/002 | Train/Val Loss: 0.01
Epoch: 006/010 | Batch 001/002 | Train/Val Loss: 0.21
Epoch: 007/010 | Batch 000/002 | Train/Val Loss: 0.10
Epoch: 007/010 | Batch 001/002 | Train/Val Loss: 0.00
Epoch: 008/010 | Batch 000/002 | Train/Val Loss: 0.08
Epoch: 008/010 | Batch 001/002 | Train/Val Loss: 0.07
Epoch: 009/010 | Batch 000/002 | Train/Val Loss: 0.01
Epoch: 009/010 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 010/010 | Batch 000/0

In [53]:
model.eval()

with torch.no_grad():
    outputs = model(X_train)

print(outputs)

tensor([[ 4.7226, -2.3685],
        [ 4.1373, -2.0483],
        [ 3.3138, -1.5980],
        [-1.3824,  0.8305],
        [-1.3824,  0.8305]])


In [54]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([[    0.9992,     0.0008],
        [    0.9979,     0.0021],
        [    0.9927,     0.0073],
        [    0.0986,     0.9014],
        [    0.0986,     0.9014]])
tensor([0, 0, 0, 1, 1])


In [55]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [56]:
predictions == y_train


tensor([True, True, True, True, True])

In [57]:
torch.sum(predictions == y_train)


tensor(5)

In [58]:
def compute_accuracy(model, dataloader):

    model = model.eval()
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        
        with torch.no_grad():
            logits = model(features)
        
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [59]:
compute_accuracy(model, train_loader)


1.0

In [60]:
compute_accuracy(model, test_loader)


1.0

In [61]:
# saving the model
torch.save(model.state_dict(), "model.pth")


In [64]:
model = NeuralNetwork(2, 2, 8,4) # needs to match the original  model exactly
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>